### Netstat Connection Locations

#### See locations for IPs that your computer is connected to.

##### Requirements:
- ipinfo.io API key - Free tier but registration required - https://ipinfo.io/account/token
- Google API key with billing - $300 credit for 90 days and cheap after - https://developers.google.com/maps/get-started



In [ ]:
import pandas as pd
import re
import requests
import json
from ipywidgets import interact, widgets
from IPython.display import Image
from io import BytesIO
import googlemaps
from googlemaps.maps import StaticMapMarker


pd.set_option('display.max_rows', 500)

##### Run command in terminal to copy netstat to clipboard:
netstat -n | clip

###### Paste in netstat input field in next cell

In [ ]:
ipinfo_input = widgets.Text(
 placeholder='paste ipinfo token',
 description='ipinfo token'
 )

google_input = widgets.Text(
 placeholder='paste google token',
 description='google token'
 )

netstat_input = widgets.Textarea(
 value='',
 placeholder='paste netstat',
 description='netstat'
 )


display(ipinfo_input, google_input, netstat_input)





In [ ]:
row_list = netstat_input.value.split('\n\n')[1].splitlines()

netstat_df = pd.DataFrame([row.split() for row in row_list[1:]], columns=['Proto', 'Local Address', 'Foreign Address', 'State'])

netstat_df

In [ ]:
netstat_df['Remote IP'] = netstat_df['Foreign Address'].apply(lambda x: x.split(':')[0] if '[' not in x else x.split(']:')[0].strip('['))

priv4 = re.compile("^(?:10|127|172\.(?:1[6-9]|2[0-9]|3[01])|192\.168)\..*")
priv6 = re.compile("::1")

netstat_df['Remote IP Type'] = netstat_df['Remote IP'].apply(lambda x: 'private' if (re.match(priv4, x) or re.match(priv6, x)) else 'public')

remote_public_df = netstat_df.loc[netstat_df['Remote IP Type']=='public', ['Remote IP']].drop_duplicates()['Remote IP'].sort_values().to_frame().reset_index(drop=True)

rmt_ip_cnt = len(remote_public_df)
print(f'{str(rmt_ip_cnt)} remote connections in netstat')

In [ ]:
data = []
headers = {'Accept': 'application/json'}
print('getting ip info')
for ip in remote_public_df['Remote IP']:
    print(ip)
    ip_url = f'https://ipinfo.io/{ip}?token={ipinfo_input.value}'
    res = requests.get(ip_url, headers=headers)
    data.append(json.load(BytesIO(res.content)))
print('ip info collected')
    

ip_info_df = pd.json_normalize(data)

In [ ]:
gmap_locs_df = ip_info_df['loc'].value_counts().to_frame().reset_index().rename(columns={'loc':'count', 'index':'loc'})

gmaps = googlemaps.Client(key=google_input.value)

markers = StaticMapMarker(gmap_locs_df['loc'].to_list())

remote_ip_loc_map = gmaps.static_map(size=650, markers=markers)

with open('remote_ip_loc_map.png', 'wb') as f:
    for chunk in remote_ip_loc_map:
        if chunk:
            f.write(chunk)


Image(filename='remote_ip_loc_map.png')

In [ ]:
ip_info_df